In [1]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RepeatedKFold
import numpy as np
from sklearn.model_selection import cross_validate
import re

In [40]:
filename = "../data/preprocessed_data/Mhealth_150_25.csv"

numbers = re.findall(r'\d+', filename)
window_size = numbers[0]
overlap = numbers[1] 

In [41]:
data = pd.read_csv(filename)
data.drop(["Sensor", "ID", "DTS", ], axis=1, inplace=True)

subjects = data["Subject"]

data.drop("Subject", axis=1, inplace=True)

Y_tChest = data["Activity"]
Y_tLA = data["Activity"]
Y_tRLA = data["Activity"]

data.drop("Activity", axis=1, inplace=True)

X = data[data.columns.to_list()]

X_tChest = X.filter(like='tChest')
X_tLA = X.filter(like='tLA')
X_tRLA = X.filter(like='tRLA')

X.head()

,max_Acctr_tChest_X(m/s2),max_Acctr_tChest_Y(m/s2),max_Acctr_tChest_Z(m/s2),max_Acctr_tLA_X(m/s2),max_Acctr_tLA_Y(m/s2),max_Acctr_tLA_Z(m/s2),max_Acctr_tRLA_X(m/s2),max_Acctr_tRLA_Y(m/s2),max_Acctr_tRLA_Z(m/s2),max_Gype_tLA_X(deg/s),...,var_MgFld_tLA_Y(T),var_MgFld_tLA_Y(uT),var_MgFld_tLA_Z(T),var_MgFld_tLA_Z(uT),var_MgFld_tRLA_X(T),var_MgFld_tRLA_X(uT),var_MgFld_tRLA_Y(T),var_MgFld_tRLA_Y(uT),var_MgFld_tRLA_Z(T),var_MgFld_tRLA_Z(uT)
0,-9.1065,0.69219,1.3479,2.6634,-9.2129,0.76847,-2.5823,-8.8917,2.0668,-0.19109,...,5.364091e-14,0.053641,3.131807e-14,0.031318,1.170388e-13,0.117039,3.571286e-13,0.357129,3.233093e-13,0.323309
1,-9.3725,0.58919,1.2780,2.6227,-9.1535,0.67597,-2.5602,-8.8420,2.0614,-0.19852,...,1.930683e-13,0.193068,3.045784e-14,0.030458,2.474357e-13,0.247436,1.371821e-12,1.371821,5.601447e-13,0.560145
2,-9.3312,0.75182,1.1054,2.6622,-9.2031,0.68218,-2.5695,-8.8420,2.0460,-0.19481,...,7.879689e-14,0.078797,2.689561e-14,0.026896,1.184993e-13,0.118499,6.231212e-13,0.623121,4.578484e-13,0.457848
3,-9.3337,0.75182,1.0386,2.6732,-9.2031,0.74547,-2.6130,-8.8531,2.0148,-0.19109,...,5.184063e-14,0.051841,2.450693e-14,0.024507,1.750402e-13,0.175040,8.266895e-13,0.826690,3.294642e-13,0.329464
4,-9.4184,0.66073,1.1164,2.6619,-9.2133,0.80207,-2.7111,-8.8724,1.9815,-0.20037,...,7.168525e-14,0.071685,2.143047e-14,0.021430,1.068484e-13,0.106848,3.107903e-13,0.310790,4.117430e-13,0.411743


In [42]:
max_depth_tChest = 4
max_depth_tLA = 4
max_depth_tRLA = 4
n_repeats = 30
k_folds = 10

rf = np.zeros(n_repeats)
rf_f1_tChest = np.zeros(n_repeats)
rf_wf1_tChest = np.zeros(n_repeats)
rf_f1_tLA = np.zeros(n_repeats) 
rf_wf1_tLA = np.zeros(n_repeats)
rf_f1_tRLA = np.zeros(n_repeats)
rf_wf1_tRLA = np.zeros(n_repeats)

cv = RepeatedKFold(n_splits=k_folds, n_repeats=n_repeats, random_state=0)
    
model_tChest = RandomForestClassifier(max_depth=max_depth_tChest)
model_tLA = RandomForestClassifier(max_depth=max_depth_tLA)
model_tRLA = RandomForestClassifier( max_depth=max_depth_tRLA)
    
scoring = ['f1_weighted']
    
scores_tChest = cross_validate(model_tChest, X_tChest, Y_tChest, scoring=scoring, cv=cv, n_jobs=-1)
scores_tLA = cross_validate(model_tLA, X_tLA, Y_tLA, scoring=scoring, cv=cv, n_jobs=-1)
scores_tRLA = cross_validate(model_tRLA, X_tRLA, Y_tRLA, scoring=scoring, cv=cv, n_jobs=-1)

In [43]:
rf = np.arange(1, n_repeats*k_folds + 1)
metrics_df = pd.DataFrame({
    'n_estimators': rf,
    f'weighted_f1_C_{max_depth_tChest}_{window_size}_{overlap}': scores_tChest['test_f1_weighted'],
    f'weighted_f1_L_{max_depth_tLA}_{window_size}_{overlap}': scores_tLA['test_f1_weighted'],
    f'weighted_f1_R_{max_depth_tRLA}_{window_size}_{overlap}': scores_tRLA['test_f1_weighted']
}) 

metrics_df.to_csv("D:/Estudios/Tesis/resultados/ult_exp/md/150_25/300.csv", index=False)